## Goal is to gather the day and weather info 

In [ ]:
'qwerqwer'.lower()

In [ ]:
%load_ext autoreload
%autoreload 2


# Day info 

In [ ]:
from energyclustering.data.fluvius.data import read_data_pickle
import pandas as pd

In [ ]:
info_df, data_df = read_data_pickle()

In [ ]:
years = data_df.index.get_level_values('year').unique()
years

### All the days

In [ ]:
first_year, last_year = years.min(), years.max()
all_days = pd.date_range(f"1/1/{first_year}", f"31/12/{last_year}", freq = '1D')
all_days

In [ ]:
day_df = pd.DataFrame(index = all_days)
day_df

### Standard date information

In [ ]:
day_df = day_df.assign(
    day = lambda x: x.index.day, 
    month = lambda x: x.index.month, 
    year = lambda x: x.index.year, 
    day_of_week = lambda x: x.index.weekday,
    is_weekend = lambda x: x.index.day_of_week >=5,
    iso_day = lambda x: x.index.day_of_year,
)
day_df.head()

### Season information

In [ ]:
def season_from_date(date): 
    month = date.month
    if 3 <= month <= 5: 
        return 'spring'
    elif 6<= month <=8: 
        return 'summer'
    elif 9<=month <= 11: 
        return 'autumn'
    return 'winter'

day_df['season'] = day_df.index.map(season_from_date)
day_df.head()

### Is holiday

In [ ]:
import holidays
belgium_holidays = holidays.BE()
day_df['is_holiday'] = all_days.map(lambda date: date in belgium_holidays)
day_df.head()

# World Weather Online

In [ ]:
weather_df = pd.DataFrame(index = all_days)
weather_df

In [ ]:
from wwo_hist import retrieve_hist_data
from pathlib import Path

In [ ]:
def get_weather_data_for_city(city): 
    start_day = all_days.min().replace(month = 1, day = 1)
    end_day = all_days.max()
    
    end_dates = pd.date_range(start_day,end_day, freq = '1M')
    start_dates = end_dates.map(lambda x: x.replace(day = 1))
    
    intervals_to_get = list(zip(start_dates, end_dates))
    API_KEY = '968285fd8d4b48feb52154524221702'
    location_list = [city]
    all_dfs = []
    for start_date, end_date in intervals_to_get: 
        start_str = start_date.strftime('%Y-%m-%d')
        end_str = end_date.strftime('%Y-%m-%d')
        hist_weather_data = retrieve_hist_data(
            api_key = API_KEY, 
            location_list = [city], 
            start_date = start_str, 
            end_date = end_str, 
            frequency = 24, 
            store_df = True
        )[0]
        all_dfs.append(hist_weather_data)
    total_weather_data = pd.concat(all_dfs)
    return total_weather_data
    

In [ ]:
if not Path('weather_data_brussels.pkl').exists(): 
    get_weather_data_for_city('brussels').to_pickle('weather_data_brussels.pkl')
if not Path('weather_data_aalst.pkl').exists(): 
    get_weather_data_for_city('aalst').to_pickle('weather_data_aalst.pkl')

In [ ]:
l = list(pd.read_pickle('weather_data_aalst.pkl').columns)
print(', '.join(l))
